In [ ]:
import numpy as np
import jax
import torch
import jax.numpy as jnp

from dln.data import get_Low_light_training_set

In [ ]:
train_set = get_Low_light_training_set(
        upscale_factor=1, patch_size=128, data_augmentation=True
    )

In [ ]:
def numpy_data_loader(dataset, batch_size, shuffle=True):
    indices = np.arange(len(dataset))
    if shuffle:
        np.random.shuffle(indices)
    for start_idx in range(0, len(dataset) - batch_size + 1, batch_size):
        excerpt = indices[start_idx : start_idx + batch_size]
        batch = [dataset[i] for i in excerpt]
        low_light, normal_light = zip(*batch)
        jaxed_low_light = jnp.array(low_light)
        jaxed_normal_light = jnp.array(normal_light)
        yield jnp.transpose(jaxed_low_light, (0, 2, 3, 1)), jnp.transpose(
            jaxed_normal_light, (0, 2, 3, 1)
        )

In [ ]:
for iteration, batch in enumerate(numpy_data_loader(train_set, 32), 1):
    print("iteration", iteration)
    LL_t, NL_t = batch
    print("Image Shapes")
    print(LL_t.shape, NL_t.shape)
